## Google Extract

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re 
import time 
import pandas as pd
from dotenv import load_dotenv
import os

In [7]:
url = 'https://www.google.com/search?q=reliance+industries+ltd&sca_esv=05f85961c74d2d97&biw=1536&bih=695&tbm=nws&sxsrf=ACQVn0_7KSZ_qW_oajmEair_DFkWHROl0g%3A1710530347874&ei=K5_0ZffpNPWh0-kPj7SvgAM&ved=0ahUKEwi3q4yO_vaEAxX10DQHHQ_aCzAQ4dUDCA0&uact=5&oq=reliance+industries+ltd&gs_lp=Egxnd3Mtd2l6LW5ld3MiF3JlbGlhbmNlIGluZHVzdHJpZXMgbHRkMgsQABiABBixAxiDATILEAAYgAQYsQMYgwEyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABEj8CVC8Ali3B3AAeACQAQCYAZYCoAHaB6oBBTAuMy4yuAEDyAEA-AEBmAIFoAKFCMICEBAAGIAEGIoFGEMYsQMYgwHCAggQABiABBixA8ICDRAAGIAEGIoFGEMYsQPCAgoQABiABBiKBRhDmAMAiAYBkgcFMC4zLjKgB4Uh&sclient=gws-wiz-news'
driver = webdriver.Chrome()

driver.get(url)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.1)

In [8]:
div_elements = driver.find_elements(By.CSS_SELECTOR , 'div.SoAPf')

source_list = []
content_list = []
time_list = []
source_links = []

pages = 0 
# flag = True
while( True):
    
    try:
        div_elements = driver.find_elements(By.CSS_SELECTOR , 'div.SoAPf')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        link_elements = driver.find_elements(By.CSS_SELECTOR , '.WlydOe')
        for i,element in enumerate(div_elements):
            # time.sleep(2)
            split_list = element.text.split('\n')
            time_list.append(split_list[-1])
            dot_index = split_list.index('.')
            temp_str = ''
            for j in range(1,dot_index):
                temp_str = temp_str +" " + split_list[j]
            source_list.append(split_list[0])
            content_list.append(temp_str)
            source_links.append(link_elements[i].get_attribute('href'))
        
        # Find the element with id="pnnext"
        print(len(source_list), source_list)
        next_page_element = driver.find_element(By.ID , "pnnext")
        # Extract the link
        next_page_link = next_page_element.get_attribute("href")
        # print("Link of the tag with id='pnnext':", next_page_link)
        print(next_page_link)
        driver.get(next_page_link)
        pages +=1
    except:
    
        df = pd.DataFrame({'Sources' : source_list , 'Content' : content_list , 'Time' : time_list , 'Source Links' : source_links})
        break
    

10 ['The Indian Express', 'Times of India', 'mint', 'Business Today', 'The Economic Times', 'CNBC TV18', 'MarketWatch', 'Forbes', 'Upstox', 'The Hindu']
https://www.google.com/search?q=reliance+industries+ltd&sca_esv=05f85961c74d2d97&biw=1536&bih=695&tbm=nws&ei=XbX1ZZ3BAemnvr0P7vqI4As&start=10&sa=N&ved=2ahUKEwjd44q1h_mEAxXpk68BHW49ArwQ8NMDegQIBBAW
20 ['The Indian Express', 'Times of India', 'mint', 'Business Today', 'The Economic Times', 'CNBC TV18', 'MarketWatch', 'Forbes', 'Upstox', 'The Hindu', 'Business Insider India', 'HW News English', 'Hindustan Times', 'Scroll.in', 'Reuters', 'The Indian Express', 'The Economic Times', 'mint', 'The Economic Times', 'MarketWatch']
https://www.google.com/search?q=reliance+industries+ltd&sca_esv=05f85961c74d2d97&biw=1536&bih=695&tbm=nws&ei=YbX1Zc_HGLvj1e8P7raJiAo&start=20&sa=N&ved=2ahUKEwiP_JW3h_mEAxW7cfUHHW5bAqE4ChDw0wN6BAgDEBk
30 ['The Indian Express', 'Times of India', 'mint', 'Business Today', 'The Economic Times', 'CNBC TV18', 'MarketWatch', 

In [11]:
def convert_to_hours(time_str):
    # Extract the numerical value from the string
    num = int(re.search(r'\d+', time_str).group())
    # Check the unit (hours, days, etc.) and convert to hours
    if 'hour' in time_str:
        return num
    elif 'day' in time_str:
        return num * 24  # Convert days to hours
    else:
        return None

# Convert the 'Time' column to hours
df['Time (hours)'] = df['Time'].apply(convert_to_hours)

df.to_csv('GoogleExtract_unfiltered.csv')
# Filter the rows where time is less than or equal to 1 day (24 hours)
filtered_df = df[df['Time (hours)'] <= 24]

# Display the resulting DataFrame
# print(filtered_df)

In [5]:
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.to_csv('GoogleExtract.csv')